In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9740,2024-10-22,Finlândia Korisliiga,12:30,Helsinki Seagulls,Pyrinto Tampere,1.01,9.35,167.5,1.83,1.83,-18.5,1.97,1.71,0G6nyTJ0,0.990099,0.106952,0.546448,0.546448,0.097051,130.226,16.976935,0.130365,1.2,1.643168,1.369306,132.89,1.584,0.301960,0.190631,40.0,244.844,134.798512,0.550549,0.6,1.341641,2.236068,189.12,2.998,1.820088,0.607101,-16.0,97,96,1.37,1.97,116.026,256.610,1.138469,0.000000,0.099917,-0.53,-0.106,-0.094340,0.767190,0.7,-0.067190,-2.88,-0.576,-14.496528,0.394789,0.4,0.005211
9741,2024-10-22,Finlândia Korisliiga,12:30,Bisons Loimaa,BC Nokia,1.65,2.06,159.5,1.83,1.83,-3.5,1.98,1.70,MVfeZ6lD,0.606061,0.485437,0.546448,0.546448,0.091497,192.026,90.509960,0.471342,3.0,0.000000,0.000000,311.04,2.334,1.110216,0.475671,35.0,166.776,41.757258,0.250379,1.2,1.643168,1.369306,148.77,2.084,0.645972,0.309967,-20.0,81,87,3.84,1.71,167.770,178.294,0.156288,0.000000,0.107603,-0.44,-0.088,-7.386364,0.484085,0.3,-0.184085,-1.97,-0.394,-2.690355,0.424071,0.4,-0.024071
9742,2024-10-22,Europa Eurocopa Feminina,14:05,Elitzur Ramla F,Arka Gdynia F,2.86,1.35,147.5,1.83,1.83,-6.5,1.95,1.72,WOdVyzxS,0.349650,0.740741,0.546448,0.546448,0.090391,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,121.050,74.304924,0.613837,2.4,1.341641,0.559017,95.20,1.880,1.354308,0.720377,23.0,68,80,1.10,1.19,0.000,0.000,0.507236,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9743,2024-10-22,Europa Enbl,15:30,Newcastle Eagles,Spartak Pleven,1.72,1.95,169.5,1.83,1.83,-2.5,1.99,1.70,E3vLDihK,0.581395,0.512821,0.546448,0.546448,0.094216,198.378,90.747635,0.457448,2.4,1.341641,0.559017,151.97,2.130,0.928897,0.436102,25.0,325.986,164.693685,0.505217,0.6,1.341641,2.236068,190.40,4.038,2.199527,0.544707,-33.0,91,80,1.67,2.38,0.000,0.000,0.088629,0.000000,0.111144,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9744,2024-10-22,Europa Enbl,13:00,Inter Bratislava,Legia,2.86,1.35,170.5,1.85,1.81,5.5,2.00,1.69,YTx92RqJ,0.349650,0.740741,0.540541,0.552486,0.090391,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,178.416,55.038204,0.308482,2.4,1.341641,0.559017,137.41,2.308,1.283752,0.556219,31.0,0,91,0.00,1.51,0.000,197.838,0.507236,0.015456,0.118809,0.00,0.000,inf,0.000000,0.0,0.000000,-0.27,-0.054,-6.481481,0.000000,0.0,0.000000
9745,2024-10-22,Europa Copa Europeia Da Fiba,13:30,BC Kalev,Alba Fehervar,1.27,3.62,166.5,1.81,1.89,-9.5,1.98,1.71,ljt6u77c,0.787402,0.276243,0.552486,0.529101,0.063645,243.258,127.610034,0.524587,1.8,1.643168,0.912871,159.21,3.328,1.592755,0.478592,9.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,73,1.83,3.46,0.000,0.000,0.679632,0.030578,0.103479,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9746,2024-10-22,Europa Copa Europeia Da Fiba,13:15,Rilski Sportist,Le Portel,6.69,1.09,160.5,1.81,1.89,12.5,1.98,1.71,zBExHLUb,0.149477,0.917431,0.552486,0.529101,0.066908,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,61,104,1.92,1.29,0.000,0.000,1.017943,0.030578,0.103479,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9743,15:30,Europa Enbl,Newcastle Eagles,Spartak Pleven,1.72,Back Home
9768,08:35,China Cba,Beijing,Xinjiang,1.73,Back Home
9775,23:00,Eua Nba,Los Angeles Lakers,Minnesota Timberwolves,1.93,Back Home
